In [1]:
import os, sys, time
import torch
from IPython.display import HTML, display

from main import main
from network import modeling

In [2]:
models = {
    'deeplabv3plus_mobilenet': {'batch_size': 32, 'val_batch_size': 32},
    'deeplabv3plus_xception': {'batch_size': 16, 'val_batch_size': 16},
    'deeplabv3plus_resnet101': {'batch_size': 16, 'val_batch_size': 16},
    'segmenter_vit_base': {'batch_size': 1, 'val_batch_size': 1},
    'mae_segmenter_vit_base': {'batch_size': 2, 'val_batch_size': 2},
    'mae_segmenter_vit_huge': {'batch_size': 1, 'val_batch_size': 1}
}

In [3]:
def train(model, use_ckpt=False):
    config = models[model]
    ckpt = f"checkpoints/latest_{model}_cityscapes_os16.pth"
    command = f"python main.py --model {model} --dataset cityscapes --gpu_id 0 --total_epochs 100 --base_lr 0.1 --loss_type focal_loss --crop_size 768 --batch_size {config['batch_size']} --val_batch_size {config['val_batch_size']} --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes"
    if use_ckpt and os.path.exists(ckpt):
        command += f" --ckpt {ckpt} --continue_training"
    sys.argv = command.split()[1:]
    print("--------------------------------------------------")
    print(f"Train model {model} with command:")
    print(command)
    print("--------------------------------------------------")
    main()
    
def test(model):
    config = models[model]
    ckpt = f"checkpoints/best_{model}_cityscapes_os16.pth"
    if not os.path.exists(ckpt):
        print(f"Test model failed, cannot find trained weights {ckpt}")
    else:
        command = f"python main.py --model {model} --dataset cityscapes --gpu_id 0 --val_batch_size {config['val_batch_size']} --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes --test_only --ckpt {ckpt}"
        sys.argv = command.split()[1:]
        print("--------------------------------------------------")
        print(f"Evaluate model {model} with command:")
        print(command)
        print("--------------------------------------------------")
        main()
        performance(model, ckpt)
        
def performance(model, ckpt):
    if not torch.cuda.is_available():
        return
    torch.cuda.empty_cache()
    device = torch.device('cuda')
    m = modeling.__dict__[model](num_classes=19, output_stride=16)
    m.load_state_dict(torch.load(ckpt, map_location=torch.device('cpu'))['model_state'])
    m = m.to(device)
    m.eval()
    with torch.no_grad():
        avg_time_cost = 0.0
        for i in range(10):
            X = torch.rand((1,3,1024,2048)).to(device)
            time_cost = time.time()
            Y = m(X)
            avg_time_cost += time.time() - time_cost
    del m
    torch.cuda.empty_cache()
    avg_time_cost /= 10.0
    print(f"Time cost per input: {avg_time_cost*1000:0.1f}ms, FPS={int(1/avg_time_cost):d}")

In [4]:
user_selection = {
    'model': 'mae_segmenter_vit_base',
    'use_ckpt': 'false',
    'action': 'test'
}

dropdown_options = ""
for model in models.keys():
    dropdown_options += f'<option value="{model}">{model}</option>'

dropdown_menu = f'''
<div style="margin-bottom: 10px;">
  <label for="model">Select Model:</label>
  <select id="model">
    {dropdown_options}
  </select>
</div>
<div style="margin-bottom: 10px;">
  <label for="ckpt">Use Checkpoint:</label>
  <select id="ckpt">
    <option value="true">True</option>
    <option value="false">False</option>
  </select>
</div>
<div style="margin-bottom: 10px;">
  <label for="action">Select Action:</label>
  <select id="action">
    <option value="train">Train</option>
    <option value="test">Test</option>
    <option value="train_test">Train and Test</option>
  </select>
</div>
'''

dropdown_menu += '''
<script>
  document.getElementById('model').onchange = function() {
    IPython.notebook.kernel.execute('user_selection["model"] = "' + this.value + '"');
  };
  document.getElementById('ckpt').onchange = function() {
    IPython.notebook.kernel.execute('user_selection["use_ckpt"] = "' + this.value + '"');
  };
  document.getElementById('action').onchange = function() {
    let ckpt = document.getElementById('ckpt')
    if (this.value === 'test') {
      ckpt.value = 'false'
      ckpt.disabled = true
    } else {
      ckpt.disabled = false
    }
    IPython.notebook.kernel.execute('user_selection["action"] = "' + this.value + '"');
  };
</script>
'''

display(HTML(dropdown_menu))

In [5]:
if "train" in user_selection["action"]:
    train(user_selection["model"], user_selection["use_ckpt"]=='true')
if "test" in user_selection["action"]:
    test(user_selection["model"])

--------------------------------------------------
Evaluate model mae_segmenter_vit_base with command:
python main.py --model mae_segmenter_vit_base --dataset cityscapes --gpu_id 0 --val_batch_size 2 --use_amp --output_stride 16 --data_root ./datasets/data/cityscapes --test_only --ckpt checkpoints/best_mae_segmenter_vit_base_cityscapes_os16.pth
--------------------------------------------------
Device: cuda
Dataset: cityscapes, Train set: 2975, Val set: 500
Model restored from checkpoints/best_mae_segmenter_vit_base_cityscapes_os16.pth


250it [01:54,  2.19it/s]



Overall Acc: 0.957513
Mean Acc: 0.829954
FreqW Acc: 0.921731
Mean IoU: 0.752829

Time cost per input: 271.7ms, FPS=3
